<center> <h1>PRIMERA PREDICIÓN DEL SUEÑO DE UN PACIENTE</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>

[URL PHYSIONET](https://physionet.org/content/sleep-accel/1.0.0/)

###### Indice

* **1. Preprocesado, extracción de características.**
  *   **1.1 Extracción variables de frecuencia cardíaca**
  *  **1.2 Extracción variables de movimiento**
  *  **1.3 Obtención matrices**
* **2. Conjuntos Train y Test**
  * **2.1 Preparación datos de entreada y salida**
* **3. Clasificador SVM Binario**
   * **3.1 Creación del modelo y selección parámetros libres**
   * **3.2 Evaluación del modelo**
* **4. Clasficador SVM Multiclase**
   * **4.1 Creación del modelo y selección parámetros libres**
      *     **4.2.1**
      *     **4.2.2**
   * **4.3 Evaluación del modelo**
* **5. Clasificador multiclase primera ventana**
    * **5.1 Residuos**

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statistics as stats
from sklearn.model_selection import GridSearchCV
from random import sample
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

## 1.Preprocesado, extracción de características

In [2]:
pacientes_sueño=[]
for file in (os.listdir("labels")):
    pacientes_sueño.append(os.path.join("labels",file))
    pacientes_sueño=sorted(pacientes_sueño)
    
pacientes_frec_card=[]
for file in os.listdir("heart_rate"):
    pacientes_frec_card.append(os.path.join("heart_rate",file))

pacientes_pasos=[]
for file in os.listdir("steps"):
    pacientes_pasos.append(os.path.join("steps",file))

pacientes_movimiento=[]
for file in os.listdir("motion"):
    pacientes_movimiento.append(os.path.join("motion",file))

La siguiente celda de código se corresponde con la ventana deslizante que recorrerá la señal fisiológica deseada. La función toma como datos de entrada el tamaño de la ventana en segundos y la señal sobre la que se quiere trabajar.

### 1.1 Extracción variables de frecuencia cardíaca

In [3]:
def get_window(w_size,signal,n_sueño):
    time = signal["Tiempo"]
    f_list=[]
    
    for i in range(n_sueño):
        a = w_size*(1+i)
        b = w_size*(2+i)

        idx=[]
        for t in time:
            if t>a and t<b:
                idx.append(True)
            else:
                idx.append(False)
        
        f = signal[idx]
        f_list.append(f)
        
    return f_list

En todos los casos, se seleccionará una w_size de 30 segundos para tener una característica cada 30s y así coincidir la cantidad de los datos adquiridos por medio del Apple Watch con los datos recogidos por la PSG.

Con la señal de frecuencia cardíaca se han decidido sacar 3 vectores de características: mínimo, máximo y media.

### 1.2 Extracción variables de movimiento

### 1.3 Obtención matrices
Realizamos esta operación sobre todos los pacientes:
(TARDA UN POCO, no más de 2 minutos, con movimiento tarda MUCHÍSIMO)

In [4]:
matrix = []
for a, (i,j,k,l) in  enumerate(zip(pacientes_sueño,pacientes_frec_card,pacientes_pasos,pacientes_movimiento)):
    #SUEÑO
    sueño = pd.read_csv(i, delimiter=' ')
    sueño.columns = ["Tiempo","Etiqueta"]
    
    #SUEÑO BINARIO
    sueño_bin = sueño.replace(to_replace =[-1,5],  value =0) 
    sueño_bin = sueño_bin.replace(to_replace =[2,3,4],  value =1)    #Etiquetas binarias
    bin_labels = sueño_bin["Etiqueta"]
    
    #ID_LIST
    labels = sueño["Etiqueta"]
    id_list = [a]*len(labels)
    
    #FRECUENCIA CARDIACA
    frec_card = pd.read_csv(j)
    frec_card.columns = ["Tiempo","Frec card"]
    frec_card=frec_card[frec_card["Tiempo"]>0]
    
    w_size = 30
    signal = frec_card
    n_sueño = len(sueño)
    f_list = get_window(w_size,signal,n_sueño)

    fc_mean_list = []
    fc_min_list = []
    fc_max_list = []

    for f in f_list:
        fc_mean_list.append(round(f['Frec card'].mean(),2))
        fc_min_list.append(round(f['Frec card'].min(),2))
        fc_max_list.append(round(f['Frec card'].max(),2))
    
    feat1 = fc_min_list
    feat2 = fc_max_list
    feat3 = fc_mean_list
    
    #ACCELERACION
    mov=pd.read_csv(l, delimiter=' ')
    mov.columns=["Tiempo","Ac_x","Ac_y","Ac_z"]
    mov_t=mov[mov['Tiempo']>0]
    mov_t=mov_t.sample(frac=.1)
    mov_t=mov_t.sort_values('Tiempo')
    raiz_movto= np.sqrt(mov_t.Ac_x**2 + mov_t.Ac_y**2 + mov_t.Ac_z**2)
    mov_t['Raiz_movto'] = raiz_movto
    signal=mov_t
    n_sueño=len(sueño)
    
    move_list= get_window(w_size,signal,n_sueño)
    mean_sqrt_acc=[]
    min_accx=[]
    max_accx=[]
    mean_accx=[]
    min_accy=[]
    max_accy=[]
    mean_accy=[]
    min_accz=[]
    max_accz=[]
    mean_accz=[]
    for f in move_list:
        mean_sqrt_acc.append(round(f['Raiz_movto'].mean(),2))
        min_accx.append(round(f['Ac_x'].min(),2))
        max_accx.append(round(f['Ac_x'].max(),2))
        mean_accx.append(round(f['Ac_x'].mean(),2))
        min_accy.append(round(f['Ac_y'].min(),2))
        max_accy.append(round(f['Ac_y'].max(),2))
        mean_accy.append(round(f['Ac_y'].mean(),2))
        min_accz.append(round(f['Ac_z'].min(),2))
        max_accz.append(round(f['Ac_z'].max(),2))
        mean_accz.append(round(f['Ac_z'].mean(),2))
        
    feat4=mean_sqrt_acc
    feat5=min_accx
    feat6=max_accx
    feat7=mean_accx
    feat8=min_accy
    feat9=max_accy
    feat10=mean_accy
    feat11=min_accz
    feat12=max_accz
    feat13=mean_accz
    
    datos = {'Min_hr': feat1, "Max_hr": feat2,"Mean_hr":feat3,'Men_acc_sqrt': feat4,
             "Min_acc_x": feat5,"Max_acc_x":feat6,"Mean_acc_x": feat7,
             "Min_acc_y": feat8,"Max_acc_y":feat9,"Mean_acc_y":feat10,
             "Min_acc_z": feat11,"Max_acc_z":feat12,"Mean_acc_z":feat13,
             "Etiquetas":labels,"Etiquetas binarias":bin_labels,"ID_P":id_list}
    datos_paciente = pd.DataFrame(data=datos)
    matrix.append(datos_paciente)

#### FEATURES

* **feat1: min_list**. La primera característica que se extrae es la lista de frecuencias mínimas.
* **feat2: max_list**. La segunda característica que se extrae es la lista de frecuencias máximas.
* **feat3: mean_list**. La tercera característica que se extrae es la lista de frecuencias medias.


* **Queremos añadir el movimiento, la raíz cuadrada de la suma de sus 3 componentes (eje x, y, z)** --> problema velocidad--> ¿paralelizar?


#### OUTCOME

* **labels:**. Lista de etiquetas del sueño

#### IDENTIFICACION
* **ID:**. Variable que identifica a cada paciente.
¿Porqué valores NaN?

In [5]:
#matrix

In [6]:
print(len(matrix))

31


In [7]:
#matrix[0][-5:]

Como hay valores NaN en algunas de las matrices, se utiliza la siguiente funcion para limpiar el dataframe de Nan y valores infinitos:

In [8]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [9]:
clean_matrix = clean_dataset(pd.concat(matrix))

## 2. Conjuntos Train y Test

In [44]:
lista = list(range(31)) # Lista de 31 numeros
pac_train = sample(lista,k=21) # 21 pacientes para train
pac_test = [el for el in lista if el not in pac_train] #Los que no estén en train que vayan a test (10)

print(len(pac_train),'pacientes de train:',pac_train)
print(len(pac_test),'pacientes de test:',pac_test)

21 pacientes de train: [17, 15, 23, 0, 12, 29, 20, 2, 26, 10, 24, 6, 13, 25, 7, 16, 28, 3, 19, 21, 11]
10 pacientes de test: [1, 4, 5, 8, 9, 14, 18, 22, 27, 30]


In [45]:
#print(len(clean_matrix)) #¿¿¿LONGITUD VARIABLE Y LONGITUD DF???? DISTINTOS
#clean_matrix

In [46]:
train_matr = []
test_matr = []
id_paciente = clean_matrix['ID_P']

for i in pac_train:    
    idx = []
    for d in id_paciente:
        if d==i:
            idx.append(True)
        else:
            idx.append(False)
    m = clean_matrix[idx]
    train_matr.append(m)

for i in pac_test:    
    idx = []
    for d in id_paciente:
        if d==i:
            idx.append(True)
        else:
            idx.append(False)
    m = clean_matrix[idx]
    test_matr.append(m)

### 2.1 Preparación datos de entrada y salida

In [47]:
# Obtengo 10 ventanas significativas de cada paciente
lista_ventanas=list(np.linspace(0,0.9,10)) #[0,1/10,1/5,3/10,4/10,5/10,6/10,7/10,8/10,9/10]
Train = pd.DataFrame()  
for i in range(len(train_matr)):
    lista_multi=[int(element * len(train_matr[i])) for element in lista_ventanas]
    ven1=train_matr[i].iloc[lista_multi]
    Train=Train.append(ven1)

Test = pd.DataFrame()  
for i in range(len(test_matr)):
    lista_multi=[int(element * len(test_matr[i])) for element in lista_ventanas]
    ven2=test_matr[i].iloc[lista_multi]
    Test=Test.append(ven2)


In [48]:
Train

,Min_hr,Max_hr,Mean_hr,Men_acc_sqrt,Min_acc_x,Max_acc_x,Mean_acc_x,Min_acc_y,Max_acc_y,Mean_acc_y,Min_acc_z,Max_acc_z,Mean_acc_z,Etiquetas,Etiquetas binarias,ID_P
0,79.0,86.0,81.17,1.00,-0.62,1.09,0.69,-0.30,1.10,0.39,-1.01,0.89,-0.00,-1.0,0.0,17.0
98,58.0,66.0,61.67,0.99,-0.70,-0.69,-0.70,-0.18,-0.17,-0.17,-0.69,-0.68,-0.69,2.0,1.0,17.0
196,68.0,71.0,69.60,1.00,-0.40,-0.38,-0.39,0.44,0.46,0.45,-0.81,-0.79,-0.80,2.0,1.0,17.0
294,64.0,84.0,74.50,1.01,-0.21,0.32,0.03,-1.03,0.05,-0.33,-0.00,1.14,0.89,2.0,1.0,17.0
392,63.0,72.0,66.17,0.99,-0.48,-0.44,-0.45,0.00,0.11,0.09,-0.89,-0.85,-0.88,0.0,0.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,52.0,54.0,53.00,1.00,-0.65,-0.65,-0.65,-0.56,-0.55,-0.56,0.51,0.51,0.51,2.0,1.0,11.0
587,50.0,55.0,51.33,1.00,-0.45,-0.45,-0.45,-0.71,-0.70,-0.71,0.53,0.54,0.54,5.0,0.0,11.0
685,52.0,54.0,53.43,1.00,-0.49,-0.49,-0.49,-0.71,-0.71,-0.71,0.49,0.50,0.50,3.0,1.0,11.0
783,56.0,59.0,57.67,0.99,-0.32,-0.31,-0.32,0.15,0.17,0.17,-0.93,-0.92,-0.93,5.0,0.0,11.0


In [49]:
Test

,Min_hr,Max_hr,Mean_hr,Men_acc_sqrt,Min_acc_x,Max_acc_x,Mean_acc_x,Min_acc_y,Max_acc_y,Mean_acc_y,Min_acc_z,Max_acc_z,Mean_acc_z,Etiquetas,Etiquetas binarias,ID_P
40,69.0,75.0,72.80,1.00,-0.04,0.02,-0.01,-0.36,-0.12,-0.21,-1.00,-0.94,-0.97,0.0,0.0,1.0
132,62.0,62.0,62.00,1.00,0.07,0.08,0.08,-0.44,-0.43,-0.43,-0.90,-0.89,-0.90,3.0,1.0,1.0
225,68.0,69.0,68.67,1.00,0.78,0.79,0.78,-0.04,-0.04,-0.04,-0.62,-0.61,-0.62,3.0,1.0,1.0
318,72.0,75.0,73.17,1.00,-0.10,-0.10,-0.10,0.42,0.43,0.42,-0.91,-0.90,-0.90,2.0,1.0,1.0
410,71.0,72.0,71.20,1.00,-0.30,-0.30,-0.30,0.11,0.12,0.11,-0.95,-0.95,-0.95,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,51.0,53.0,52.00,0.99,-0.28,-0.27,-0.27,-0.45,-0.43,-0.44,-0.85,-0.84,-0.85,2.0,1.0,30.0
430,48.0,49.0,48.50,0.99,-0.31,-0.29,-0.30,-0.15,-0.14,-0.14,-0.94,-0.93,-0.94,2.0,1.0,30.0
501,65.0,67.0,65.33,0.99,-0.34,-0.32,-0.33,-0.44,-0.43,-0.43,-0.84,-0.82,-0.83,5.0,0.0,30.0
572,52.0,52.0,52.00,1.00,0.40,0.42,0.41,-0.01,0.00,-0.01,-0.92,-0.90,-0.91,2.0,1.0,30.0


In [50]:
Train.to_csv(r'C:\Users\rober\OneDrive\Escritorio\pRocesado_ordena\Train.csv')
Test.to_csv(r'C:\Users\rober\OneDrive\Escritorio\pRocesado_ordena\Test.csv')